# Паттерн цепочка обязанностей

### Создаём класс машин

Сначала определим класс, который будет обрабатывать цепочкой обязанностей: 

In [ ]:
class Car:
    def __init__(self):
        self.name = "CAR"  # Название
        self.km = 11100    # Пробег
        self.fuel = 5      # Количество бензина, %
        self.oil = 5       # Количество масла, %

### События

Создаём список событий

In [2]:
E_FUEL, E_KM, E_OIL = "E_FUEL", "E_KM", "E_OIL"

Создаём класс событий, который будет хранить всю ииформацию о событии (в данном случае только тип)

In [3]:
class Event:
    def __init__(self, kind):
        self.kind = kind

### Обработчики событий

Пустой обработчик

In [4]:
class NullHandler:
    def __init__(self, successor=None):
        self.__successor = successor

    def handle(self, car, event):
        if self.__successor is not None:
            self.__successor.handle(car, event)

Debug обработчик - выводит информацию о принятом событии и передаёт дальше по цепочке

In [ ]:
class DebugHandler(NullHandler):

    def handle(self, car, event):
        print("*START_EVENT*: {}".format(event.kind))
        super().handle(car, event)

Если запрашиваеться событие `E_FUEL`, проводиться проверка количества бензина и, при необходимости, заливаеться полный бак

In [6]:
class FuelHandler(NullHandler):
    def handle(self, car, event):
        if event.kind == E_FUEL:
            if car.fuel < 10:
                print("Added fuel")
                car.fuel = 100
        else:
            super().handle(car, event)

Если запрашиваеться событие `E_KM`, проводиться проверка километража и, при необходимости, обнуляеться

In [7]:
class KmHandler(NullHandler):
    def handle(self, car, event):
        if event.kind == E_KM:
            if car.km > 10000:
                print("Made a car test.")
                car.km = 0
        else:
            super().handle(car, event)

Если запрашиваеться событие `E_OIL`, проводиться проверка количества масла и, при необходимости, заливаеться

In [8]:
class OilHandler(NullHandler):
    def handle(self, car, event):
        if event.kind == E_OIL:
            if car.oil < 10:
                print("Added oil")
                car.oil = 100
        else:
            super().handle(car, event)

### Автосервис

При создании нового автосервиса, создаём две цепочки:
1. простая, обрабатывающая по очереди:
  * не надо ли проверить бензин;
  * не надо ли проверить масло;
  * не надо ли проверить километраж.
2. простая цепочка, дополненная выводом информации о запросе.

In [9]:
class CarService:
    def __init__(self):
        self.handlers = FuelHandler(OilHandler(KmHandler(NullHandler())))
        self.handlers_D = DebugHandler(self.handlers)
        self.events = []

    def add_event(self, event):
        self.events.append(event)

    def handle_car(self, car):
        for event in self.events:
            self.handlers.handle(car, event)

    def handle_car_D(self, car):
        for event in self.events:
            self.handlers_D.handle(car, event)

### Проверка

In [10]:
def test():
  # определяем задания
    events = [Event(E_FUEL), Event(E_KM), Event(E_OIL)]

  # создаём автосервис
    service = CarService()

  # Добавляем задания автосервису
    for event in events:
        service.add_event(event)

    print("=== NORMAL ===")
    # Выполняем все задания
    car = Car()
    service.handle_car(car)
    print("--------------")
    car.fuel = 5
    service.handle_car(car)
    print("--------------")
    car.oil = 5
    service.handle_car(car)
    print("--------------")
    car.km = 20000
    service.handle_car(car)

    print("=== DEBUG ===")

    # Выполняем все задания с выводом запросов
    car = Car()
    service.handle_car_D(car)
    print("--------------")
    car.fuel = 5
    service.handle_car_D(car)
    print("--------------")
    car.oil = 5
    service.handle_car_D(car)
    print("--------------")
    car.km = 20000
    service.handle_car_D(car)


test()

=== NORMAL ===
Added fuel
Made a car test.
Added oil
--------------
Added fuel
--------------
Added oil
--------------
Made a car test.
=== DEBUG ===
*START_EVENT*: E_FUEL
Added fuel
*START_EVENT*: E_KM
Made a car test.
*START_EVENT*: E_OIL
Added oil
--------------
*START_EVENT*: E_FUEL
Added fuel
*START_EVENT*: E_KM
*START_EVENT*: E_OIL
--------------
*START_EVENT*: E_FUEL
*START_EVENT*: E_KM
*START_EVENT*: E_OIL
Added oil
--------------
*START_EVENT*: E_FUEL
*START_EVENT*: E_KM
Made a car test.
*START_EVENT*: E_OIL


### Дополнение

Естественно, обработчики событий не обязаны быть процедурами: они могут возвращать како-нибудь результат. Такой подход очень хорошо использовать для вычленения объекта из очереди. Такую цепочку хорошо использовать в следующей задаче:

>**Дана база работников фирмы. Необходимо сделать автоматическую систему распределения заданий по рабочим.**

> В таком случае проще всего устроить цепочку рабочих, в которую передаётся параметры задачи. Результатом работы цепочки будет работник, с соответствующей компетенцией и достаточно малой загруженностью или же передаётся на рассмотрение следующему работнику.